In [ ]:
import os
from glob import glob
import re
from pickle import load

import pandas as pd
import numpy as np
from matplotlib import pyplot

from settings.settings import HISTORIC_DATA_FOLDER

%config IPCompleter.greedy=True
%matplotlib inline

file_list = glob('climate_month_*.pickle', root_dir=HISTORIC_DATA_FOLDER)
day_list = glob('climate_20*-*-*.pickle', root_dir=HISTORIC_DATA_FOLDER)

In [ ]:
def get_year_and_month_from_file_name(file_name):
    month = re.search('202\d-\d\d', file_name).group()
    return month

In [ ]:
def get_daily_average_temperature(df_day):
    try:
        return df_day['Temp'].mean()
    except TypeError:
        assert df_day is None
        return None

In [ ]:
def add_week_number_column(df_in, year):
    df_in = df_in.loc[year]
    df_in['Week Number'] = df_in.index.isocalendar().week
    df_in.set_index('Week Number', inplace=True)
    
    return df_in

In [ ]:
all_weather = {
    'date': [],
    'av_temp': []
}

for name in sorted(file_list):
    yr_month = get_year_and_month_from_file_name(name)
    with open(os.path.join(HISTORIC_DATA_FOLDER, name), 'rb') as fh:
        month_dict = load(fh)
    for key in month_dict.keys():
        date_as_string = f'{yr_month}-{key:02}'
        all_weather['date'].append(date_as_string)
        all_weather['av_temp'].append(get_daily_average_temperature(month_dict[key]))

In [ ]:
df_temp = pd.DataFrame.from_dict(all_weather)

dt_concat_format = '%d-%m-%Y %H:%M:%S'
df_temp['date'] = pd.to_datetime(df_temp['date'], format='%Y-%m-%d')
df_temp.set_index('date', inplace=True)

In [ ]:
ax = df_temp.plot(grid=True, title='Daily average temperature.')
ax.set_ylabel('Temperature (°C)')

In [ ]:
temp_per_week = df_temp.resample('w').mean(numeric_only=True)
av_temp_per_week_2021 = add_week_number_column(temp_per_week, '2021')
av_temp_per_week_2022 = add_week_number_column(temp_per_week, '2022')

# last week of 2021 is according to pandas in the year 2022. Move it to 2021, to prevent weird plotting.
av_temp_per_week_2021.loc[52] = av_temp_per_week_2022.loc[52]
av_temp_per_week_2022.drop(52, inplace=True)

In [ ]:
fig, axes = pyplot.subplots(figsize=(30, 10))
ax = av_temp_per_week_2021['av_temp'].plot(label='per week 2021', grid=True)
bx = av_temp_per_week_2022['av_temp'].plot(label='per week 2022', grid=True,
                                           title='Weekly average temperature.', style='o-')
bx.set_ylabel('Temperature (°C)')
axes.legend()
pyplot.show()

In [ ]:
per_hour = list()
for name in sorted(file_list):
    yr_month = get_year_and_month_from_file_name(name)
    with open(os.path.join(HISTORIC_DATA_FOLDER, name), 'rb') as fh:
        month_dict = load(fh)
    for day, value in month_dict.items():
        try:
            per_hour.append(value)
        except AttributeError:
            print(f'No dataframe for {yr_month}-{day:02}')

##### Append uncompleted pickled months

for name in sorted(day_list):
    yr_m_d = re.search('202\d-\d\d-\d\d', name).group()

    with open(os.path.join(HISTORIC_DATA_FOLDER, name), 'rb') as fh:
        df_day_hour = load(fh)

    try:
        per_hour.append(df_day_hour)
    except AttributeError:
        print(f'No dataframe in file {name}')

temperature_per_hour = pd.concat(per_hour)
temperature_per_hour.head()

In [ ]:
last_x_hrs = 120
ax = temperature_per_hour[-last_x_hrs:]['Temp'].plot(title=f'Hourly temperature, last {last_x_hrs} hours.')
ax.set_ylabel('Temperature (°C)')

In [ ]:
temp_pd = temperature_per_hour.resample('d').mean(numeric_only=True)
temp_pw = temperature_per_hour.resample('w').mean(numeric_only=True)

In [ ]:
fig, axes = pyplot.subplots(figsize=(40, 8), sharex=True)
ax = temp_pd['Temp'].plot(grid=True, title='Average temperature', label='per day')
bx = temp_pw['Temp'].plot(label='per week', grid=True, style='o-')

axes.set_ylabel('Temperature (°C)')
axes.set_xlabel('Date')
axes.legend()
pyplot.show()

In [ ]:
temp_pw_2023 = add_week_number_column(temp_pw, '2023')
temp_pw_2022 = add_week_number_column(temp_pw, '2022')
temp_pw_2021 = add_week_number_column(temp_pw, '2021')

# last week of 2021 is according to pandas in the year 2022. Move it to 2021, to prevent weird plotting.
temp_pw_2021.loc[52] = temp_pw_2022.loc[52]
temp_pw_2022.drop(52, inplace=True)

# first week of 2023 is according to pandas in the year 2022. Move it back, to prevent weird plotting.
temp_pw_2022.loc[52] = temp_pw_2023.loc[52]
temp_pw_2023.drop(52, inplace=True)

In [ ]:
# fig, axes = pyplot.subplots(sharex=True, figsize=(40, 8))
# ax = temp_pw_2021['Temp'].plot(label='2021', grid=True, style='d--')
# bx = temp_pw_2022['Temp'].plot(label='2022', grid=True, style='o-')
# cx = temp_pw_2023['Temp'].plot(label='2023', grid=True, style='cx-')
# cx.set_ylabel('Temperature (°C)')

# axes.set_title('Average temperature - year by year comparison')
# axes.legend()
# pyplot.show()

In [ ]:
from pickle import load

gas_export_file_name = 'gas_per_week.pickle'

with open(gas_export_file_name, 'rb') as fh:
    gpw_export = load(fh)

In [ ]:
# fig, axes = pyplot.subplots(sharex=True, figsize=(40, 8))
# ax = gpw_export['2021']['Volume'].plot(label='2021', grid=True, style='d--')
# bx = gpw_export['2022']['Volume'].plot(label='2022', grid=True, style='o-')
# cx = gpw_export['2023']['Volume'].plot(label='2023', grid=True, style='cx-')

# cx.set_ylabel('gas volume (m³)')
# axes.set_title('Gas volume per week - year by year comparison')
# axes.legend()
# pyplot.show()

In [ ]:
fig, axes = pyplot.subplots(2, 1, sharex=True, figsize=(40, 16))
gpw_export['2021']['Volume'].plot(ax=axes[0], label='gas 2021', grid=True, style='d--')
gpw_export['2022']['Volume'].plot(ax=axes[0], label='gas 2022', grid=True, style='o-')
gpw_export['2023']['Volume'].plot(ax=axes[0], label='gas 2023', grid=True, style='cx-')
axes[0].set_ylabel('gas volume (m³)')
axes[0].set_title('Gas volume per week - year by year comparison')
axes[0].legend()

temp_pw_2021['Temp'].plot(ax=axes[1], label='tmp 2021', grid=True, style='d--')
temp_pw_2022['Temp'].plot(ax=axes[1], label='tmp 2022', grid=True, style='o-')
temp_pw_2023['Temp'].plot(ax=axes[1], label='tmp 2023', grid=True, style='cx-')
axes[1].set_ylabel('Temperature (°C)')
axes[1].set_title('Average temperature - year by year comparison')
axes[1].legend()

pyplot.show()